In [5]:
import os
import random
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def ffd(items, B):
    items = sorted(items, reverse=True)
    bins = []
    for x in items:
        placed = False
        for i in range(len(bins)):
            if bins[i] >= x:
                bins[i] -= x
                placed = True
                break
        if not placed:
            bins.append(B - x)
    return len(bins)

def exact_binpacking(a, B):
    n = len(a)
    N = 1 << n
    INF = 10**9
    dp = [INF]*N
    load = [0]*N
    dp[0] = 1
    for mask in range(N):
        if dp[mask] == INF:
            continue
        for i in range(n):
            if not (mask & (1<<i)):
                nm = mask | (1<<i)
                if load[mask] + a[i] <= B:
                    if dp[nm] > dp[mask]:
                        dp[nm] = dp[mask]
                        load[nm] = load[mask] + a[i]
                else:
                    if dp[nm] > dp[mask] + 1:
                        dp[nm] = dp[mask] + 1
                        load[nm] = a[i]
    return dp[-1]

def generate_instance(n, B, seed=None):
    random.seed(seed)
    # Generate train usages uniformly between 1 and B/2 to avoid trivial bins
    return [random.randint(1, B//2) for _ in range(n)]

def run_experiments(out_dir="mcla_experiments", seed=42):
    os.makedirs(out_dir, exist_ok=True)
    random.seed(seed)
    np.random.seed(seed)

    n_values = list(range(8, 18))  # n from 8 to 17 for exact DP feasibility
    B = 100

    results = []
    for n in n_values:
        for trial in range(10):  # 10 trials per n
            items = generate_instance(n, B)
            # Run exact DP (time it)
            start = time.time()
            exact_bins = exact_binpacking(items, B)
            exact_time = time.time() - start

            # Run FFD heuristic (time it)
            start = time.time()
            ffd_bins = ffd(items, B)
            ffd_time = time.time() - start

            approx_ratio = ffd_bins / exact_bins if exact_bins > 0 else np.nan

            results.append({
                "n": n,
                "trial": trial,
                "exact_bins": exact_bins,
                "ffd_bins": ffd_bins,
                "approx_ratio": approx_ratio,
                "exact_time": exact_time,
                "ffd_time": ffd_time
            })

    df = pd.DataFrame(results)
    csv_path = os.path.join(out_dir, "experiment_results.csv")
    df.to_csv(csv_path, index=False)

    # Plot 1: Approximation ratio mean + median vs n
    plt.figure(figsize=(8,6))
    grouped = df.groupby("n")["approx_ratio"]
    x = []
    y_mean = []
    y_median = []
    for n, series in grouped:
        x.append(n)
        y_mean.append(series.mean())
        y_median.append(series.median())
    plt.plot(x, y_mean, marker='o', label='Mean ratio')
    plt.plot(x, y_median, marker='x', label='Median ratio')
    plt.xlabel("Number of trains (n)")
    plt.ylabel("Approximation ratio (FFD / Exact)")
    plt.title("FFD Approximation Ratio vs Exact DP")
    plt.legend()
    plt.grid(True)
    ratio_vs_n = os.path.join(out_dir, "ratio_vs_n.png")
    plt.savefig(ratio_vs_n)
    plt.close()

    # Plot 2: Approximation ratio boxplot per n with fixed labels
    plt.figure(figsize=(8,6))
    n_values = sorted(df["n"].unique())
    data_to_plot = [df[df["n"] == n]["approx_ratio"].dropna() for n in n_values]
    plt.boxplot(data_to_plot, labels=n_values, patch_artist=True)
    plt.xlabel("Number of trains (n)")
    plt.ylabel("Approximation ratio (FFD / Exact)")
    plt.title("FFD Approximation Ratio Boxplot")
    plt.grid(True)
    plt.xticks(ticks=range(1, len(n_values) + 1), labels=n_values)
    plt.tight_layout()
    approx_ratio_boxplot = os.path.join(out_dir, "approx_ratio_boxplot.png")
    plt.savefig(approx_ratio_boxplot)
    plt.close()

    # Plot 3: Runtime comparison (mean) vs n
    plt.figure(figsize=(8,6))
    grouped_time = df.groupby("n")[["exact_time", "ffd_time"]].mean()
    plt.plot(grouped_time.index, grouped_time["exact_time"], marker='o', label="Exact DP")
    plt.plot(grouped_time.index, grouped_time["ffd_time"], marker='x', label="FFD heuristic")
    plt.xlabel("Number of trains (n)")
    plt.ylabel("Runtime (seconds)")
    plt.title("Runtime Comparison")
    plt.legend()
    plt.grid(True)
    runtime_comparison = os.path.join(out_dir, "runtime_comparison.png")
    plt.savefig(runtime_comparison)
    plt.close()

    # Plot 4: Histogram of approximation ratios for max n
    max_n = max(n_values)
    plt.figure(figsize=(8,6))
    approx_ratios = df[df["n"] == max_n]["approx_ratio"].dropna()
    plt.hist(approx_ratios, bins=10, edgecolor='black')
    plt.xlabel("Approximation ratio (FFD / Exact)")
    plt.ylabel("Frequency")
    plt.title(f"Approximation Ratio Histogram (n={max_n})")
    plt.grid(True)
    ratio_histogram = os.path.join(out_dir, "ratio_histogram.png")
    plt.savefig(ratio_histogram)
    plt.close()

    # Plot 5: Density sweep (vary B with fixed n)
    n_fixed = 14
    B_values = list(range(50, 201, 10))
    density_results = []
    for B_curr in B_values:
        ratios = []
        for _ in range(5):  # 5 trials each
            items = generate_instance(n_fixed, B_curr)
            exact = exact_binpacking(items, B_curr)
            ffd_res = ffd(items, B_curr)
            if exact > 0:
                ratios.append(ffd_res / exact)
        density_results.append(np.mean(ratios) if ratios else np.nan)
    plt.figure(figsize=(8,6))
    plt.plot(B_values, density_results, marker='o')
    plt.xlabel("Capacity B")
    plt.ylabel("Mean Approximation Ratio")
    plt.title(f"Density Sweep (n={n_fixed})")
    plt.grid(True)
    density_sweep = os.path.join(out_dir, "density_sweep.png")
    plt.savefig(density_sweep)
    plt.close()

    print(f"Wrote outputs to: {out_dir}")
    print(f"CSV: {csv_path}")
    print("Plots:", ratio_vs_n, approx_ratio_boxplot, runtime_comparison, ratio_histogram, density_sweep)

if __name__ == "__main__":
    run_experiments()


Wrote outputs to: mcla_experiments
CSV: mcla_experiments\experiment_results.csv
Plots: mcla_experiments\ratio_vs_n.png mcla_experiments\approx_ratio_boxplot.png mcla_experiments\runtime_comparison.png mcla_experiments\ratio_histogram.png mcla_experiments\density_sweep.png
